In [1]:
# This section of the code is for all of my imports
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pandas as pd
from datetime import datetime
import glob
import matplotlib.cm as cm

# This allows me to access my flash drive that im working off of
os.chdir('/Volumes/Flash Drive')

In [ ]:
# This section loads all CSV files from the directory

Path = '/Volumes/Flash Drive/Saturns rings Research/Data From Center of Ringlets CSV files/Center Median Ringlets'

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(Path, '*.csv'))

print(f"Found {len(csv_files)} CSV files:")
for csv_file in csv_files:
    print(f"  - {os.path.basename(csv_file)}")

In [3]:
# This Section is the formulas to convert from UTC to ET time, im working off of ET time

# This formula gets the UTC and ET offset due to leap seconds, number is in seconds
def utc_to_et_offset(year):
    """
    Get the offset between UTC and ET (TDB) in seconds for 2008.
    
    For 2008:
    - Leap seconds accumulated by 2008: 33 seconds
    - TT-TAI offset: 32.184 seconds
    - ET ≈ TDB ≈ TT for most purposes
    - So ET - UTC ≈ 33 + 32.184 = 65.184 seconds
    """
    
    ls = 33  # Leap Seconds since 2008

    # TT - TAI offset is always 32.184 seconds
    tt_tai_offset = 32.184
    
    # ET ≈ TDB ≈ TT = UTC + leap_seconds + 32.184
    et_utc_offset = ls + tt_tai_offset
    
    return et_utc_offset # Seconds

# This section of the code converts the UTC Julian Date to ET Julian Date adding in the leap seconds offset
def convert_paper_time_to_et(jd_utc):
    """
    Convert the paper's UTC-based Julian Date to ET-based Julian Date.
    
    Parameters:
    -----------
    jd_utc : float
        Julian Date in UTC (as used in the paper)
    
    Returns:
    --------
    jd_et : float
        Julian Date in Ephemeris Time
    """
    
    # Get offset for 2008
    et_utc_offset_2008 = utc_to_et_offset(2008)
    
    # Convert to ET
    initial_time = jd_utc + (et_utc_offset_2008 / 86400.0)
    
    return initial_time # Seconds


In [4]:
# This section defines the True Anamoly formula and the radius formula

def calculate_radius_true_anomaly(a, e, true_anomaly):
    """
    Calculate the radius at a given true anomaly for a Keplerian ellipse.
    
    This implements equation (2) from the paper:
    r(λ,t) = a(1 - e²) / (1 + e·cos(f))
    f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
    
    Parameters:
    -----------
    a : float
        Semi-major axis (km)
    e : float
        Eccentricity (dimensionless, between 0 and 1)
    true_anomaly : float or array
        True anomaly f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
        where λ is the inertial longitude and ϖ is the longitude of periapse
    
    Returns:
    --------
    r : float or array
        Radius at the given true anomaly (km)
    """

        # Convert to radians
    #true_anomaly = true_anomaly * np.pi / 180
    
    # Calculate the numerator: a(1 - e²)
    numerator = a * (1 - e**2)
    
    # Calculate the denominator: 1 + e·cos(f)
    denominator = 1 + e * np.cos(true_anomaly)
    
    # Calculate radius
    r = numerator / denominator
    
    return r

def calculate_true_anomaly(longitude #Inertial longitude (LON value)
                           ,varpi_0 # Longitude periapse (Fixed)
                           ,varpi_dot #Rrecession rate (Fixed)
                           ,time # Time from data
                           ,initial_time): # Time from paper (fixed)
    """
    Calculate the true anomaly from orbital parameters.
    
    From the paper: f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
    
    Parameters:
    -----------
    longitude : float or array
        Inertial longitude λ (degrees)
    varpi_0 (Longitude periapse) : float
        Longitude of periapse at epoch ϖ₀ (degrees)
    varpi_dot (precession rate) : float, optional
        Apsidal precession rate ϖ̇ (degrees/day)
    time : float, optional
        Current time (days)
    initial_time : float, optional
        Epoch time t₀ (days)
    
    Returns:
    --------
    true_anomaly : float or array
        True anomaly f (degrees)
    """
    
    # True anomaly is the angle from periapse
    true_anomaly = longitude - varpi_0 - varpi_dot * (time - initial_time)

    # Wrap to 0-360 degrees
    true_anomaly = true_anomaly % 360
    
    return true_anomaly

def plot_r_vs_true_anomaly(true_anomaly, radii, title=f"\Radius vs True Anomaly Analysis of Titan Ringlet \n File: {csv_files}"):
    """
    Plot radius vs true anomaly.
    
    Parameters:
    -----------
    true_anomaly : array
        True anomaly values in degrees
    radii : array
        Radius values in km
    title : str
        Plot title
    """
    plt.figure(figsize=(10, 6))
    plt.plot(true_anomaly, radii, 'b-', linewidth=2)
    plt.xlabel('True Anomaly, f (degrees)')
    plt.ylabel('Radius (km)')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# This section defines static parameters and initializes data storage

# J2000 epoch = JD 2451545.0
j2000_jd = 2451545.0

# Paper's epoch
paper_epoch_utc = 2454467.0  # This is in UTC

# Paper's epoch conversion to ET (seconds)
paper_epoch_jd_et = convert_paper_time_to_et(paper_epoch_utc)

# Static orbital parameters/Paper values (these don't change between occultations)
# Using median values for center of ringlet
a_paper = 77878.67  # km
e_paper = 2.88*10**-4

# Median parameters (averaged from inner and outer edge values)
aeI = 17.39    # ae Inner
aeO = 27.20    # ae Outer
ae_median = (aeI + aeO) / 2  # Median ae

varpi_0I = 270.54  # degrees Inner
varpi_0O = 270.70  # degrees Outer
varpi_0_median = (varpi_0I + varpi_0O) / 2  # Median longitude of periapse

varpi_dotI = 22.57503  # ϖ̇ in degrees/day Inner
varpi_dotO = 22.57562  # ϖ̇ in degrees/day Outer
varpi_dot_median = (varpi_dotI + varpi_dotO) / 2  # Median precession rate

# Initialize lists to store data points from all occultations
# Single lists for median/center data
all_true_anomaly = []
all_radii = []
all_longitudes = []
all_times = []
all_TAUPLUS = []
all_TAUMINUS = []
all_csv_names = []

In [ ]:
# This section loops through all CSV files and processes each one for Titan Ringlet
print(f"\nProcessing {len(csv_files)} CSV files...\n")

for csv_file in csv_files:
    csv_name = os.path.basename(csv_file)
    print(f"Processing: {csv_name}")
    
    try:
        # Load data from CSV
        data = pd.read_csv(csv_file)
        
        # Get Titan median/center data (not inner/outer)
        titan_data = data[data['Ringlet_Position'] == 'Titan']
        
        if len(titan_data) == 0:
            print(f"  ⚠ No Titan ringlet data found in {csv_name}")
            continue
        
        # Process Median/Center Data
        # Extract radius value from CSV
        a_median = titan_data['Radius'].values[0]
        
        # Calculate eccentricity (using median ae from paper then using our radial value)
        e_median = ae_median / a_median
        
        # Get longitude from data
        longitude_median = titan_data['LON'].values[0]  # LON in degrees

        # Get TAUPLUS from data
        TAUPLUS_median = titan_data['TAUPLUS'].values[0]  # TAUPLUS

        # Get TAUMINUS from data
        TAUMINUS_median = titan_data['TAUMINUS'].values[0]  # TAUMINUS
        
        # Convert ET time from seconds to days
        titan_et_days_median = titan_data['ET'].values[0] / 86400.0
        
        # Convert to Julian Days
        titan_jd_et_median = j2000_jd + titan_et_days_median
        
        # Calculate true anomaly (comes out in degrees after wrapping)
        true_anomaly_median = calculate_true_anomaly(longitude_median, varpi_0_median, varpi_dot_median, 
                                             titan_jd_et_median, paper_epoch_jd_et)
        
        # Convert to radians before calculating radius
        true_anomaly_rad_median = true_anomaly_median * np.pi / 180
        
        # Calculate radii
        radii_median = calculate_radius_true_anomaly(a_median, e_median, true_anomaly_rad_median)
        
        # Store median/center data points
        all_true_anomaly.append(true_anomaly_median)
        all_radii.append(radii_median)
        all_longitudes.append(longitude_median)
        all_times.append(titan_jd_et_median)
        all_TAUPLUS.append(TAUPLUS_median)
        all_TAUMINUS.append(TAUMINUS_median)
        
        print(f"  ✓ Center/Median: a = {a_median:.2f} km")
        
        all_csv_names.append(csv_name)
        
    except Exception as ex:
        print(f"  ✗ Error processing {csv_name}: {ex}")
        import traceback
        traceback.print_exc()
        continue

# Convert lists to numpy arrays
all_true_anomaly = np.array(all_true_anomaly)
all_radii = np.array(all_radii)
all_longitudes = np.array(all_longitudes)
all_times = np.array(all_times)
all_TAUPLUS = np.array(all_TAUPLUS)
all_TAUMINUS = np.array(all_TAUMINUS)

print(f"\n{'='*60}")
print(f"TOTAL DATA POINTS COLLECTED:")
print(f"  Center/Median: {len(all_true_anomaly)} points")
print(f"  Total Tau+- values: {len(all_TAUPLUS) + len(all_TAUMINUS)} values")
print(f"{'='*60}")

if len(all_true_anomaly) > 0:
    print(f"Center/Median - True anomaly: {np.min(all_true_anomaly):.2f}° to {np.max(all_true_anomaly):.2f}°")
    print(f"Center/Median - Radii: {np.min(all_radii):.2f} to {np.max(all_radii):.2f} km")

In [7]:
""" # This part creates the model curve using paper parameters (Might not need commenting out for now)

# Create model curve (sampling all true anomalies)
true_anomaly_deg_model = np.linspace(0, 360, 1000)
true_anomaly_rad_model = true_anomaly_deg_model * np.pi / 180

# Paper parameters for model
a_paper = 77867.13  # km
ae_paper = 17.39    # km 
e_paper = ae_paper / a_paper

model_radii = calculate_radius_true_anomaly(a_paper, e_paper, true_anomaly_rad_model)

print(f"Model parameters:")
print(f"  a_paper = {a_paper} km")
print(f"  e_paper = {e_paper:.6f}")
print(f"  Model radius range: {np.min(model_radii):.2f} to {np.max(model_radii):.2f} km")
"""

' # This part creates the model curve using paper parameters (Might not need commenting out for now)\n\n# Create model curve (sampling all true anomalies)\ntrue_anomaly_deg_model = np.linspace(0, 360, 1000)\ntrue_anomaly_rad_model = true_anomaly_deg_model * np.pi / 180\n\n# Paper parameters for model\na_paper = 77867.13  # km\nae_paper = 17.39    # km \ne_paper = ae_paper / a_paper\n\nmodel_radii = calculate_radius_true_anomaly(a_paper, e_paper, true_anomaly_rad_model)\n\nprint(f"Model parameters:")\nprint(f"  a_paper = {a_paper} km")\nprint(f"  e_paper = {e_paper:.6f}")\nprint(f"  Model radius range: {np.min(model_radii):.2f} to {np.max(model_radii):.2f} km")\n'

In [8]:
# Initial approximate locations of the ringlets (Hardcoded)
saturn_ringlets_approx = {
    'Titan': 77883,      # Colombo Gap - looks accurate in the plot around 77500-78000
    'Maxwell': 87500,    # Maxwell Gap - appears correct around 87000-87500
    'Bond': 88710,       # Bond Ringlet - looks close, around 88500-89000
    'Huygens': 117800,   # Huygens Gap - in the inset, appears around 117500
    'Dawes': 90210       # Dawes Ringlet - appears around 90000-90500
}

# Path to the directory containing all occultation files
data_directory = '/Volumes/Flash Drive/Saturns rings Research/Data/BetCen Occultations'

# Find all .sav files matching the pattern
occultation_files = glob.glob(os.path.join(data_directory, 'BetCen_*_1km.sav'))

# Sort the files for consistent ordering
occultation_files.sort()

print(f"Found {len(occultation_files)} occultation files:")
for file in occultation_files:
    print(f"  - {os.path.basename(file)}")

# Dictionary to store all extracted data
all_occultation_data = {}

# Loop through each occultation file
for file_path in occultation_files:
    # Extract the occultation identifier (e.g., '064E', '105I', etc.)
    filename = os.path.basename(file_path)
    occultation_id = filename.split('_')[1]  # Gets '064E' from 'BetCen_064E_1km.sav'
    
    print(f"\nProcessing {filename}...")
    
    try:
        # Read the .sav file
        test = sio.readsav(file_path)
        
        # Extract the data structure
        pdsdata = test['pdsdata'][0]
        
        # Get radius and tau
        radius = pdsdata['RADIUS']
        tau = pdsdata['TAU']
        
        # Store the data in the dictionary
        all_occultation_data[occultation_id] = {
            'filename': filename,
            'radius': radius,
            'tau': tau,
            'full_data': pdsdata  # Store full data structure if you need other fields
        }
        
        print(f"  ✓ Successfully extracted data: {len(radius)} data points")
        
    except Exception as e:
        print(f"  ✗ Error processing {filename}: {e}")

print(f"\n{'='*60}")
print(f"Extraction complete! Total occultations loaded: {len(all_occultation_data)}")
print(f"{'='*60}")

# Example: Loop through all occultations
for occ_id, data in all_occultation_data.items():
    print(f"{occ_id}: {len(data['radius'])} data points")

Found 12 occultation files:
  - BetCen_075I_1km.sav
  - BetCen_077E_1km.sav
  - BetCen_077I_1km.sav
  - BetCen_078E_1km.sav
  - BetCen_081I_1km.sav
  - BetCen_085I_1km.sav
  - BetCen_089I_1km.sav
  - BetCen_092E_1km.sav
  - BetCen_096I_1km.sav
  - BetCen_102I_1km.sav
  - BetCen_104E_1km.sav
  - BetCen_104I_1km.sav

Processing BetCen_075I_1km.sav...
  ✓ Successfully extracted data: 72021 data points

Processing BetCen_077E_1km.sav...
  ✓ Successfully extracted data: 70179 data points

Processing BetCen_077I_1km.sav...
  ✓ Successfully extracted data: 71561 data points

Processing BetCen_078E_1km.sav...
  ✓ Successfully extracted data: 86554 data points

Processing BetCen_081I_1km.sav...
  ✓ Successfully extracted data: 78863 data points

Processing BetCen_085I_1km.sav...
  ✓ Successfully extracted data: 70303 data points

Processing BetCen_089I_1km.sav...
  ✓ Successfully extracted data: 70032 data points

Processing BetCen_092E_1km.sav...
  ✓ Successfully extracted data: 103898 data po

In [ ]:
# This section plots the model curve with ALL data points from all occultations for Titan ringlet

plt.figure(figsize=(14, 8))

# Plot median/center data points with error bars (blue circles)
plt.errorbar(all_true_anomaly, all_radii - a_paper, 
             yerr=[all_TAUMINUS, all_TAUPLUS],  # Asymmetric error bars
             fmt='bo', markersize=8, capsize=3, capthick=1, 
             label=f'Titan Center/Median ({len(all_true_anomaly)} points)', 
             alpha=0.6, elinewidth=1)

# Formatting
plt.xlabel('True Anomaly, f (degrees)', fontsize=12)
plt.ylabel('r - a (km)', fontsize=12)
plt.title(f'Titan Ringlet - True Anomaly Analysis (Center/Median)\nCombined data from {len(all_csv_names)} occultations', fontsize=14)
plt.xlim(0, 360)
plt.ylim(-50, 70)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='a (semi-major axis)')
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)

# Add text box with statistics
stats_text = f'Total points: {len(all_true_anomaly)}\n'
stats_text += f'Occultations: {len(all_csv_names)}'

plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes, 
         fontsize=9, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nPlot generated with {len(all_true_anomaly)} total data points from {len(all_csv_names)} CSV files")
print(f"  - Center/Median: {len(all_true_anomaly)} points (blue circles)")

In [ ]:
# This section plots optical depth vs true anomaly for Titan ringlet along with the true anomaly vs radial distance
fig, ax1 = plt.subplots(figsize=(14, 8))

# PRIMARY AXIS (LEFT Y-AXIS): r - a (km)
ax1.set_xlabel('True Anomaly, f (degrees)', fontsize=12)
ax1.set_ylabel('r - a (km)', fontsize=12)
ax1.set_xlim(0, 360)
ax1.set_ylim(-50, 70)

# Plot radius data on primary axis
ax1.errorbar(all_true_anomaly, all_radii - a_paper,
             fmt='bo', markersize=8, capsize=3, capthick=1, 
             label=f'Titan Center/Median ({len(all_true_anomaly)} points)', 
             alpha=0.6, elinewidth=1)

# SECONDARY AXIS (RIGHT Y-AXIS): Optical Depth (τ)
ax2 = ax1.twinx()
ax2.set_ylabel('Optical Depth (τ)', fontsize=12, color='purple')
ax2.tick_params(axis='y', labelcolor='purple')
ax2.set_ylim(-.1, 5)

# Calculate average tau
avg_tau = [(tau_plus + tau_minus) / 2 for tau_plus, tau_minus in zip(all_TAUPLUS, all_TAUMINUS)]

# Plot optical depth data on secondary axis
ax2.errorbar(all_true_anomaly, avg_tau,
             fmt='ms', markersize=8, capsize=3, capthick=1, 
             label=f'Titan Center/Median Optical Depth ({len(all_true_anomaly)} points)', 
             alpha=0.6, elinewidth=1)

# Formatting
ax1.set_title(f'Titan Ringlet - True Anomaly Analysis (Center/Median)\nCombined data from {len(all_csv_names)} occultations', fontsize=14)
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='a (semi-major axis)')
ax1.grid(True, alpha=0.3)

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, fontsize=10, loc='best')

# Add text box with statistics
stats_text = f'Total points: {len(all_true_anomaly)}\n'
stats_text += f'Occultations: {len(all_csv_names)}'
ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, 
         fontsize=9, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

In [ ]:
# This section plots just optical depth vs true anomaly for Titan ringlet
plt.figure(figsize=(14, 8))

# Calculate average tau (mean of upper and lower bounds)
avg_tau = [(tau_plus + tau_minus) / 2 for tau_plus, tau_minus in zip(all_TAUPLUS, all_TAUMINUS)]

# Error bars: distance from mean to bounds
tau_err_lower = [avg - tau_minus for avg, tau_minus in zip(avg_tau, all_TAUMINUS)]
tau_err_upper = [tau_plus - avg for tau_plus, avg in zip(all_TAUPLUS, avg_tau)]

# Plot median/center data with asymmetric error bars
plt.errorbar(all_true_anomaly, avg_tau, 
             yerr=[tau_err_lower, tau_err_upper],
             fmt='bs', markersize=8, capsize=3, capthick=1, 
             label=f'Titan Center/Median Optical Depth ({len(all_true_anomaly)} points)', 
             alpha=0.6, elinewidth=1)

# Formatting
plt.xlabel('True Anomaly, f (degrees)', fontsize=12)
plt.ylabel('Optical Depth (τ)', fontsize=12)
plt.title(f'Titan Ringlet - Optical Depth vs True Anomaly (Center/Median)\nCombined data from {len(all_csv_names)} occultations', fontsize=14)
plt.xlim(0, 361)
plt.ylim(0, 5)  # Adjust based on your tau range
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)

# Add text box with statistics
stats_text = f'Total points: {len(all_true_anomaly)}\n'
stats_text += f'Occultations: {len(all_csv_names)}'
plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes, 
         fontsize=9, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nPlot generated with {len(all_true_anomaly)} total data points from {len(all_csv_names)} CSV files")
print(f"  - Center/Median: {len(all_true_anomaly)} points (blue squares)")

print("\n" + "=" * 60)

# Sort by true anomaly to see the pattern more clearly
data_sorted = sorted(zip(all_true_anomaly, avg_tau))

print("\nCENTER/MEDIAN (sorted by true anomaly):")
for f, tau in data_sorted:
    print(f"  f = {f:6.1f}°  ->  τ = {tau:.3f}")